In [23]:
# def data_process(prcAll):
#     """
#     Convert raw price data into closeprice columns 
#     """
#     closePrice = []
#     for stock in range(50):
#         closePrice.extend(prcAll[stock])

#     dates = []
#     for stock in range(50):
#         for day in range(prcAll.shape[1]):
#             dates.append(day)

#     stocks = []
#     for stock in range(50):
#         stocks.extend([stock]*prcAll.shape[1])
#     full_data = pd.DataFrame({'date': dates, 'stock': stocks, 'closePrice': closePrice})

#     return full_data

In [13]:
def range_so_far(data):
    amp = []
    for j in range(50):
        single_stock_data = data[j]

        base_range = np.max(single_stock_data[-AMP_WINDOW:]) -  np.min(single_stock_data[-AMP_WINDOW:])
        amp.append(base_range)
    return amp

In [57]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


SHORT_TERM = 3              # Short term average
LONG_TERM = 30              # Long term average
PRICE_RANGE = 5             # The period to calculate price difference
AMP_WINDOW = 100            # The period to get stock amplitude

INCREASE_HOLDING = 500      # Price to increase holding
DECREASE_HOLDING = 500      # Price to decrease holding
AMP_LO_THRESHOLD = 10       # The threshold when price movement is considered low
AMP_HI_THRESHOLD = 1.5      # The threshold when price movement is considered high
MSE_THRESHOLD = 0.02        # The threshold to control price volatility
SLOPE_THRESHOLD = 0.4       # The threshold to control LR slope

nInst = 50
currentPos = np.zeros(nInst)

def getMyPosition(prcSoFar):
	
    day = prcSoFar.shape[1]

    global currentPos

    currentPrices = prcSoFar[:,-1] # price of last day

    amp = range_so_far(prcSoFar)
	
    # Get long term and short term average prices
    for stock in range(50):
        single_stock_data = prcSoFar[stock]

        # Use short term and long term average to determine sign
        long_mean = single_stock_data[-LONG_TERM:].mean()
        short_mean = single_stock_data[-SHORT_TERM:].mean()
        today_sign = np.sign(short_mean - long_mean)

        # Use a price window to make decision
        n_day_diff = single_stock_data[-PRICE_RANGE] - single_stock_data[-1]

        # Calculate the MSE of price movement during the range
        n_day_gap = np.diff(single_stock_data[-PRICE_RANGE:])
        LR = LinearRegression(n_jobs=-1).fit(np.array(range(PRICE_RANGE-1)).reshape(-1, 1), n_day_gap.reshape(-1,1))
        n_day_mse = mean_squared_error(n_day_gap, LR.predict(np.array(range(PRICE_RANGE-1)).reshape(-1, 1)))
        
        if np.abs(n_day_diff) <= amp[stock]/AMP_LO_THRESHOLD or (n_day_mse > MSE_THRESHOLD and (np.abs(LR.coef_) < SLOPE_THRESHOLD)[0][0]):
            pass
            
        elif np.abs(n_day_diff) >= amp[stock]/AMP_HI_THRESHOLD:
            value = today_sign * DECREASE_HOLDING
            currentPos[stock] -= value//currentPrices[stock]
    
        else:
            value = today_sign * INCREASE_HOLDING
            currentPos[stock] += value//currentPrices[stock]
	
    return currentPos

In [60]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
# from main import getMyPosition as getPosition

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    #df=pd.read_csv(fn, sep='\s+', names=cols, header=None, index_col=0)
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./data/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

currentPos = np.zeros(nInst)

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(500,750): 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] #prcHist[:,t-1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        clipPos = np.clip(newPosOrig, -posLimits, posLimits)
        newPos = np.array([np.trunc(x) for x in clipPos])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return plmu - 0.1*plstd

calcPL(prcAll)

Loaded 50 instruments for 750 days
Day 500 value: -5.54 todayPL: $-5.54 $-traded: 5537 return: -0.00100
Day 501 value: -18.32 todayPL: $-12.79 $-traded: 11072 return: -0.00165
Day 502 value: -21.27 todayPL: $-2.95 $-traded: 19707 return: -0.00108
Day 503 value: -68.36 todayPL: $-47.10 $-traded: 26213 return: -0.00261
Day 504 value: -86.34 todayPL: $-17.97 $-traded: 31655 return: -0.00273
Day 505 value: -78.28 todayPL: $8.06 $-traded: 37170 return: -0.00211
Day 506 value: -109.46 todayPL: $-31.18 $-traded: 45212 return: -0.00242
Day 507 value: -46.82 todayPL: $62.64 $-traded: 51819 return: -0.00090
Day 508 value: -94.68 todayPL: $-47.86 $-traded: 57794 return: -0.00164
Day 509 value: -111.43 todayPL: $-16.75 $-traded: 60763 return: -0.00183
Day 510 value: -123.44 todayPL: $-12.01 $-traded: 66670 return: -0.00185
Day 511 value: -170.16 todayPL: $-46.72 $-traded: 73153 return: -0.00233
Day 512 value: -100.23 todayPL: $69.93 $-traded: 78185 return: -0.00128
Day 513 value: -185.60 todayPL: 

43.019322134972505